# Necessary Imports

In [11]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from implicit.als import AlternatingLeastSquares
from sklearn.metrics.pairwise import cosine_similarity
from Data_Splitter import build_data, create_user_track_matrix
from Fairness_Metrics import compute_recGap, compute_compounding_factor

# ALS Helper Functions

In [12]:
def get_recommendations_for_user_als(model, user_id, user_indices, item_indices, 
                                   user_track_matrix, track_list, top_n=10):
    """
    Generate recommendations for a user using the trained ALS model.
    
    Parameters:
        model: Trained ALS model
        user_id: ID of the user to generate recommendations for
        user_indices: Mapping from user IDs to matrix indices
        item_indices: Mapping from track IDs to matrix indices
        user_track_matrix: User-item interaction matrix
        track_list: List of all track IDs
        top_n: Number of recommendations to generate
        
    Returns:
        List of recommended track IDs
    """
    if user_id not in user_indices:
        return []
    
    # Get the user's index in the model
    user_idx = user_indices[user_id]
    
    # Get the user's interaction history to exclude already interacted items
    if user_id in user_track_matrix.index:
        user_history = set(user_track_matrix.loc[user_id][lambda row: row == 1].index)
    else:
        user_history = set()
    
    # Get recommendations from the model
    # The recommend function returns a list of (item_id, score) tuples
    recommendations = model.recommend(
        userid=user_idx, 
        user_items=csr_matrix(np.zeros((1, len(item_indices)))), 
        N=top_n+len(user_history), 
        filter_already_liked_items=False
    )
    
    # Convert item indices back to track IDs and filter out items in user history
    rec_track_ids = []
    for item_idx, _ in recommendations:  # Unpack properly as (item_idx, score)
        # Get the track_id from item_indices by finding the key for the given value
        reverse_item_indices = {v: k for k, v in item_indices.items()}
        if item_idx in reverse_item_indices:
            track_id = reverse_item_indices[item_idx]
            if track_id not in user_history:
                rec_track_ids.append(track_id)
                if len(rec_track_ids) >= top_n:
                    break
                
    return rec_track_ids

# Evaluation

In [ ]:
def compute_diversity_for_list(recommended_tracks, sparse_item_matrix, track_list):
    """
    Compute intra-list diversity: average dissimilarity among all pairs of recommended tracks.
    Dissimilarity is defined as (1 - cosine similarity) for each pair.
    """
    if len(recommended_tracks) < 2:
        return 0.0

    # Retrieve indices for the recommended tracks from track_list.
    indices = [track_list.index(t) for t in recommended_tracks if t in track_list]
    
    if len(indices) < 2:
        return 0.0
    
    # Extract the corresponding item vectors from the sparse matrix.
    vectors = sparse_item_matrix[indices]
    
    # Compute pairwise cosine similarity.
    sim_matrix = cosine_similarity(vectors)
    
    # Compute average pairwise similarity (ignoring the diagonal)
    sum_similarity = 0.0
    count = 0
    n = len(indices)
    for i in range(n):
        for j in range(i+1, n):
            sum_similarity += sim_matrix[i, j]
            count += 1
    
    avg_similarity = sum_similarity / count if count > 0 else 0.0
    # Diversity is defined as the complement of similarity.
    return 1 - avg_similarity

def ndcg_at_k(relevances, k):
    """
    Compute NDCG@k given a list of binary relevance scores.
    """
    relevances = np.asfarray(relevances)[:k]
    if relevances.size == 0:
        return 0.0
    # Discount factors (log2-based)
    discounts = np.log2(np.arange(2, relevances.size + 2))
    dcg = np.sum(relevances / discounts)
    # Ideal DCG (sorted relevances)
    ideal_relevances = np.sort(relevances)[::-1]
    idcg = np.sum(ideal_relevances / discounts)
    return dcg / idcg if idcg > 0 else 0.0

def evaluate_als_metrics(als_model, df, holdout_df, user_track_matrix, sparse_item_matrix, 
                        track_list, user_indices, item_indices, top_n=10):
    """
    Evaluate recommendations for all users in the holdout set using Recall@10, Coverage@10, and Diversity@10.
    Also computes metrics for each gender subgroup.
    """
    # Mapping from user_id to their ground truth track_ids.
    user_holdout = holdout_df.groupby('user_id')['track_id'].apply(set).to_dict()
    # Mapping from user_id to gender.
    user_gender = df.set_index('user_id')['gender'].to_dict()
    
    recall_scores = {}
    diversity_scores = {}
    ndcg_scores = {}
    # For coverage per gender, maintain a set of recommended items per gender.
    coverage_by_gender = {}
    
    for user, ground_truth in user_holdout.items():
        if user not in user_indices:
            continue
            
        recs = get_recommendations_for_user_als(als_model, user, user_indices, item_indices, 
                                                user_track_matrix, track_list, top_n=top_n)
        
        # Skip if no recommendations were generated
        if not recs:
            continue
        
        # Compute Recall@10.
        if ground_truth:
            recall = len(set(recs).intersection(ground_truth)) / len(ground_truth)
        else:
            recall = 0.0
        recall_scores[user] = recall
        
        # Compute NDCG@10
        relevances = [1 if rec in ground_truth else 0 for rec in recs]
        ndcg = ndcg_at_k(relevances, top_n)
        ndcg_scores[user] = ndcg
        
        # Compute Diversity@10.
        diversity = compute_diversity_for_list(recs, sparse_item_matrix, track_list)
        diversity_scores[user] = diversity
        
        # Collect recommended items per gender for Coverage.
        gender = user_gender.get(user, 'unknown')
        if gender not in coverage_by_gender:
            coverage_by_gender[gender] = set()
        coverage_by_gender[gender].update(recs)
    
    # Calculate overall metrics
    overall_recall = np.mean(list(recall_scores.values())) if recall_scores else 0.0
    overall_diversity = np.mean(list(diversity_scores.values())) if diversity_scores else 0.0
    overall_ndcg = np.mean(list(ndcg_scores.values())) if ndcg_scores else 0.0
    
    # Calculate coverage
    all_recommended_items = set().union(*(coverage_by_gender.values())) if coverage_by_gender else set()
    overall_coverage = len(all_recommended_items) / len(track_list) if track_list else 0.0
    
    # Compute per-gender averages.
    recall_by_gender = {}
    diversity_by_gender = {}
    ndcg_by_gender = {}
    coverage_metrics_by_gender = {}
    
    # Organize per-user metrics by gender.
    for user, rec in recall_scores.items():
        gender = user_gender.get(user, 'unknown')
        if gender not in recall_by_gender:
            recall_by_gender[gender] = []
        recall_by_gender[gender].append(rec)
    
    for user, div in diversity_scores.items():
        gender = user_gender.get(user, 'unknown')
        if gender not in diversity_by_gender:
            diversity_by_gender[gender] = []
        diversity_by_gender[gender].append(div)
    
    for user, ndcg in ndcg_scores.items():
        gender = user_gender.get(user, 'unknown')
        if gender not in ndcg_by_gender:
            ndcg_by_gender[gender] = []
        ndcg_by_gender[gender].append(ndcg)
    
    for gender, rec_set in coverage_by_gender.items():
        coverage_metrics_by_gender[gender] = len(rec_set) / len(track_list)
    
    # Calculate averages by gender
    avg_recall_by_gender = {g: np.mean(scores) for g, scores in recall_by_gender.items()}
    avg_diversity_by_gender = {g: np.mean(scores) for g, scores in diversity_by_gender.items()}
    avg_ndcg_by_gender = {g: np.mean(scores) for g, scores in ndcg_by_gender.items()}

    # Print metrics
    print("\nEvaluation Metrics @ {}:".format(top_n))
    print("\nOverall Recall: {:.4f}".format(overall_recall))
    print("Recall by gender:", avg_recall_by_gender)

    print("\nOverall Coverage: {:.4f}".format(overall_coverage))
    print("Coverage by gender:", coverage_metrics_by_gender)

    print("\nOverall Diversity: {:.4f}".format(overall_diversity))
    print("Diversity by gender:", avg_diversity_by_gender)
    
    print("\nOverall NDCG: {:.4f}".format(overall_ndcg))
    print("NDCG by gender:", avg_ndcg_by_gender)
    
    gender_metrics = {
        'recall': avg_recall_by_gender,
        'coverage': coverage_metrics_by_gender,
        'diversity': avg_diversity_by_gender,
        'ndcg': avg_ndcg_by_gender
    }
    
    return overall_recall, overall_coverage, overall_diversity, overall_ndcg, gender_metrics

def grid_search_als(df, user_track_matrix, sparse_item_matrix, track_list, df_val_holdout, 
                    user_indices, item_indices, factors_list, regularization_list, iterations=15):
    """
    Perform grid search over factors and regularization parameters for ALS model on validation set.
    Returns the best hyperparameters (those that achieve the highest overall NDCG).
    """
    best_ndcg = -1.0
    best_params = None
    grid_results = []  # Store tuples: (factors, regularization, overall_ndcg)
    
    # Convert sparse_item_matrix to CSR format for ALS
    user_item_matrix_csr = csr_matrix((len(user_indices), len(item_indices)))
    for user_id, user_idx in user_indices.items():
        if user_id in user_track_matrix.index:
            user_history = user_track_matrix.loc[user_id]
            for track_id in user_history[user_history == 1].index:
                if track_id in item_indices:
                    item_idx = item_indices[track_id]
                    user_item_matrix_csr[user_idx, item_idx] = 1
    
    for factors in factors_list:
        for reg in regularization_list:
            print(f"\nTraining ALS model with factors={factors}, regularization={reg}")
            # Train ALS model with current hyperparameters
            als_model = AlternatingLeastSquares(factors=factors, regularization=reg, 
                                                iterations=iterations, random_state=42)
            
            try:
                als_model.fit(user_item_matrix_csr)
                
                # Evaluate on validation set
                _, _, _, overall_ndcg, _ = evaluate_als_metrics(
                    als_model, df, df_val_holdout, user_track_matrix, 
                    sparse_item_matrix, track_list, user_indices, item_indices
                )
                
                grid_results.append((factors, reg, overall_ndcg))
                print(f"Factors: {factors}, Regularization: {reg} => NDCG: {overall_ndcg:.4f}")
                
                if overall_ndcg > best_ndcg:
                    best_ndcg = overall_ndcg
                    best_params = (factors, reg)
            except Exception as e:
                print(f"Error with factors={factors}, reg={reg}: {e}")
                continue

    if best_params is None:
        print("No successful parameter combination found. Using default values.")
        best_params = (factors_list[0], regularization_list[0])
    else:
        print("\nBest hyperparameters (factors, regularization):", best_params)
        print("Best overall NDCG on validation set:", best_ndcg)
    
    return best_params, best_ndcg, grid_results

def recGap_ALS_results(df, gender_metrics):
    for key, value in gender_metrics.items():
        print(f"\nFor the {key} metric")
        compute_recGap(value)
        compute_compounding_factor(df, value)

# Main Functions

In [14]:
def build_and_evaluate_als(df):
    # Split data into train, validation, and test sets
    print("Building data splits...")
    df_model_train, df_val_holdout, df_test_holdout = build_data(df)
    
    # Create user-track matrix and get sparse item matrix
    print("Creating user-track matrix...")
    user_track_matrix, sparse_item_matrix, track_list = create_user_track_matrix(df_model_train)
    
    # Create user and item indices for ALS
    print("Creating user and item indices...")
    unique_users = df_model_train['user_id'].unique()
    unique_tracks = track_list
    
    user_indices = {user_id: idx for idx, user_id in enumerate(unique_users)}
    item_indices = {track_id: idx for idx, track_id in enumerate(unique_tracks)}
    
    # Convert to CSR matrix for ALS
    print("Converting to CSR matrix...")
    user_item_matrix_csr = csr_matrix((len(user_indices), len(item_indices)))
    for user_id, user_idx in user_indices.items():
        if user_id in user_track_matrix.index:
            user_history = user_track_matrix.loc[user_id]
            for track_id in user_history[user_history == 1].index:
                if track_id in item_indices:
                    item_idx = item_indices[track_id]
                    user_item_matrix_csr[user_idx, item_idx] = 1
    
    print(f"Matrix shape: {user_item_matrix_csr.shape}")
    print(f"Non-zero entries: {user_item_matrix_csr.nnz}")
    
    # Define hyperparameter search space
    factors_list = [50, 100]  # Reduced for quicker execution
    regularization_list = [0.01, 0.1]  # Reduced for quicker execution
    
    # Perform grid search on validation set
    # print("\nStarting grid search...")
    # best_params, _, _ = grid_search_als(
    #     df, user_track_matrix, sparse_item_matrix, track_list, 
    #     df_val_holdout, user_indices, item_indices, 
    #     factors_list, regularization_list
    # )
    
    best_factors, best_reg = 50, 0.01
    
    # Train final ALS model with best hyperparameters
    print(f"\nTraining final ALS model with factors={best_factors}, regularization={best_reg}")
    final_als_model = AlternatingLeastSquares(
        factors=best_factors, 
        regularization=best_reg,
        iterations=15,
        random_state=42
    )
    final_als_model.fit(user_item_matrix_csr)
    
    # Evaluate on test set
    print("\nEvaluating on test set...")
    overall_recall, overall_coverage, overall_diversity, overall_ndcg, gender_metrics = evaluate_als_metrics(
        final_als_model, df, df_test_holdout, user_track_matrix, 
        sparse_item_matrix, track_list, user_indices, item_indices, top_n=10
    )
    
    # Calculate fairness metrics
    print("\nFairness Metrics:")
    recGap_ALS_results(df, gender_metrics)
    
    return final_als_model, gender_metrics

# Running the Algorithm

In [15]:
# Load the data and run the ALS pipeline.
df = pd.read_csv('data/LFM-1b-DemoBiasSub-10k.csv', header=0)
df_SMOTE = pd.read_csv('data/LFM-1b-DemoBiasSub-10k-SMOTE.csv', header=0)
df_resampled = pd.read_csv('data/LFM-1b-DemoBiasSub-10k-Resampled.csv', header=0)

In [16]:
build_and_evaluate_als(df[:10000])

Building data splits...
Creating user-track matrix...
Creating user and item indices...
Converting to CSR matrix...


c:\Users\20115\anaconda3\Lib\site-packages\scipy\sparse\_index.py:108: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])


Matrix shape: (756, 3961)
Non-zero entries: 9088

Training final ALS model with factors=50, regularization=0.01


  0%|          | 0/15 [00:00<?, ?it/s]


Evaluating on test set...


ValueError: too many values to unpack (expected 2)

In [ ]:
build_and_evaluate_als(df_SMOTE)

In [ ]:
build_and_evaluate_als(df_resampled)